In [2]:
import numpy as np # linear algebra
import pandas as pd
# from ImplicitALS import ImplicitALS as als

In [3]:
# model = als()
customers = pd.read_csv('../data/customers.csv')
articles = pd.read_csv('../data/articles.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

In [ ]:
# count = pd.read_parquet('../data/new-art-count.parquet')
# count

In [9]:
# customers
# articles
transactions

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [5]:
# df = transactions.groupby(['article_id','customer_id']).count()
# df.rename(columns={"t_dat":"count"},inplace=True)


In [8]:
# df = df["count"].reset_index()
# df

,article_id,customer_id,count
0,108775015,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,3
1,108775015,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,2
2,108775015,001ba9e81e13ce12a2585d9ebde923fe74429e9e12ea59...,1
3,108775015,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,2
4,108775015,002323971cbd38fad4512d5114676e5e17eb262db02320...,1
...,...,...,...
27306434,956217002,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,1
27306435,956217002,c89698e2626f3d98d8496e673c89ece5a8ee92914ee92d...,1
27306436,956217002,cedd533807afaef98ca626a771d8c14986de84bdb43743...,1
27306437,956217002,d51d1c7b07110ba2571d22849c1fdb7e61d7cbc1d58ada...,1


In [9]:
# atransactions = pd.merge(transactions,df )

In [10]:
# atransactions

,t_dat,customer_id,article_id,price,sales_channel_id,count
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2
1,2018-09-24,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2
2,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,1
...,...,...,...,...,...,...
31788319,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,856440002,0.042356,2,1
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1


In [12]:
from sklearn.preprocessing import LabelEncoder

ALL_USERS = customers['customer_id'].unique().tolist()
ALL_ITEMS = articles['article_id'].unique().tolist()

leUser = LabelEncoder().fit(ALL_USERS)
transactions['user_id'] = leUser.transform(transactions['customer_id'])
leItem = LabelEncoder().fit(ALL_ITEMS)
transactions['item_id'] = leItem.transform(transactions['article_id'])
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2,40179
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2,10520
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,7,6387
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,7,46304
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,7,46305
...,...,...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1371691,104961
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,1371691,100629
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1371721,104053
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1371747,88521


In [13]:
from scipy.sparse import coo_matrix

col = transactions['user_id'].values
row = transactions['item_id'].values
one = np.ones(transactions.shape[0])
# one = df["count"].to_xarray
# one = atransactions['count'].values

coo = coo_matrix((one, (row, col)), shape=(len(ALL_ITEMS), len(ALL_USERS)))

In [16]:
# coo
print( coo )

  (40179, 2)	2
  (10520, 2)	2
  (6387, 7)	1
  (46304, 7)	1
  (46305, 7)	1
  (46302, 7)	1
  (6386, 7)	1
  (47416, 198)	1
  (5944, 198)	1
  (22540, 198)	1
  (47423, 198)	1
  (47415, 198)	1
  (9226, 203)	1
  (9116, 203)	1
  (6066, 235)	2
  (6066, 235)	2
  (42957, 235)	2
  (42082, 235)	2
  (42082, 235)	1
  (31362, 235)	2
  (31362, 235)	2
  (31362, 235)	4
  (31362, 235)	4
  (44469, 235)	4
  (43407, 235)	4
  :	:
  (67539, 1370464)	1
  (93574, 1370716)	1
  (101316, 1370853)	1
  (64156, 1370853)	1
  (104148, 1370906)	1
  (98914, 1370906)	1
  (104147, 1370906)	1
  (104149, 1370906)	1
  (98913, 1370906)	1
  (104072, 1370938)	1
  (104067, 1370938)	1
  (101375, 1371091)	1
  (77917, 1371091)	1
  (77917, 1371091)	1
  (103330, 1371091)	2
  (52534, 1371091)	2
  (3521, 1371091)	1
  (88367, 1371091)	1
  (102122, 1371091)	1
  (93053, 1371091)	1
  (104961, 1371691)	1
  (100629, 1371691)	1
  (104053, 1371721)	1
  (88521, 1371747)	1
  (101689, 1371960)	1


In [17]:
csr = coo.tocsr()
csr

<105542x1371980 sparse matrix of type '<class 'numpy.int64'>'
	with 27306439 stored elements in Compressed Sparse Row format>

In [18]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

In [19]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(csr)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
from pickle import dump, load 
file = "../data/newKnn.pickle"
# dump(model_knn,open(file,"wb"))
K = load(open(file,"rb"))

KeyboardInterrupt: 

In [21]:
query_artcile = 519936007 # input article ID
# query_artcile = 891322004
# leItem.inverse_transform([query_index])
index = leItem.transform([query_artcile])
# print("Choosen article is: ",article_table.index[query_index])

In [ ]:
# csr[:, 298].toarray()

In [22]:
# distances, indices = model_knn.kneighbors(csr[index,:].toarray().reshape(1,-1), n_neighbors = 6)
distances, indices = K.kneighbors(csr[index,:].toarray().reshape(1,-1), n_neighbors = 6)

NameError: name 'K' is not defined

In [ ]:
article = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        # article.append(indices.flatten()[i])
        article.append(leItem.inverse_transform([indices.flatten()[i]]))
        distance.append(distances.flatten()[i]) 
           

m=pd.Series(article,name='article')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

# print('Recommendations for {0}:\n'.format(article_table.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["article"].iloc[i], recommend["distance"].iloc[i]))

In [24]:
x = np.array([0,3,1,0])
y = np.array([0,3,1,0])
z = np.array([1,1,1,1])
coo_matrix((z , (x,y)),shape=(4,4)).toarray()



array([[2, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]])